# **18 Чтение и запись данных. Часть 2**

Курс ведёт **Александр Джумурат** Data Scientist в ivi

## **18.1** *JSON*

JSON (англ. JavaScript Object Notation) — текстовый формат обмена данными, основанный на JavaScript. 

JSON представляет собой последовательность пар ключ-значение, ограниченных фигруными скобками, например: 
<pre>
{"firstName": "Ivan", "lastName": "Ivanov", "age": 30}
</pre>

достоинства JSON
* легко читается людьми
* занимает меньше места в текстовом виде, чем аналогичный объект в формате XML (т.к. нету таких сущностей как тэги и нет необходимости писать открывающий/закрывающий тэг)
* является "родным" форматом для Javascript, Python, MongoDB и других систем

В силу этих особенностей, этот формат стал стандартом для передачи текстовой информации в сети интернет.

В стандартный набор библиотек Python входит модуль `json`, кооторый используется для преобразования объектов (например, словарей) в текстовый JSON формат.

In [1]:
user = {
    "firstName": "Ivan",
    "lastName": "Ivanov", 
    "age": 30
}

print("Объект {} : \n{}".format(type(user), user))

Объект <class 'dict'> : 
{'firstName': 'Ivan', 'lastName': 'Ivanov', 'age': 30}


Преобразуем объект словаря в текстовый JSON формат

In [2]:
import json

user_json = json.dumps(user)
print("Объект {} : {}".format(type(user_json), user_json))

Объект <class 'str'> : {"firstName": "Ivan", "lastName": "Ivanov", "age": 30}


Готово! Мы молучили объект типа `str` который можно, например, передать по сети в другое приложение.

Обратное преобразование `str->dict` выполняется с помощью той же библиотеки JSON

In [3]:
user_dict = json.loads(user_json)
print("Объект {} : {}".format(type(user_dict), user_dict))

Объект <class 'dict'> : {'firstName': 'Ivan', 'lastName': 'Ivanov', 'age': 30}


## **18.2** *Pickle - сериализация объектов*

pickle (от англ. "маринованные огурчики") - это бинарный формат данных для хранение объектов Python. Вы можете запаковать объекты на одной машине и распаковать на другой - главное, чтобы версия питона при чтении была не ниже версии питона при записи. Pickle может сохранять простые объекты python - например, объекты numpy или словари. При дампе объектов из нестандартных библиотек (вроде sklearn) нужно гарантировать, что при распаковке объектов будут доступны те же самые библиотеки, иначе pickle может не справится.

In [4]:
import pickle 

# создаём объект python
temp_dict = {'foo': 'bar'}

print("temp_dict  -> {}".format(temp_dict))

# сохраняем объект в pickle
with open('./data/dict.pkl', 'wb') as f:
    pickle.dump(temp_dict, f)
# удаляем словарь
del temp_dict

# проверяем, что удаление прошло успешно
try:
    print("temp_dict  -> {}".format(temp_dict))
except NameError:
    print("Неудача, словарь не найден")

# загружаем сохраненный объект из pkl
with open('./data/dict.pkl', 'rb') as f:
    temp_dict = pickle.load(f)
    
print("temp_dict  -> {}".format(temp_dict))

temp_dict  -> {'foo': 'bar'}
Неудача, словарь не найден
temp_dict  -> {'foo': 'bar'}


В отличие от всех предыдущих форматов данных, которые являются текстовыми, pickle позволяет сохранять объекты python'a и загружать их непосредственно в память python'овского процессора без допольнительных преобразований из текстового вида.

## **18.3** *Работа с данными формата HDF5*

HDF5 (Hierarchical Data Format, HDF (Иерархический формат данных)) - бинарный формат, который позволяет эффективно хранить большие объемы данных. Реализация в Python имеет хорошую интеграцию с библиотекой векторных вычислений numpy. 

Основые преимущества формата:

* хранение сложных иерархических структур
* быстрый доступ к изолированным частям данных без загрузки в память (чтение с диска)

Реализация в python называется [h5py](https://www.h5py.org/). Это довольно низкоуровневая библиотека, поэтому рекомендую дополнительно изучить удобную в использовании [PyTables](https://www.pytables.org/), которая является оберткой над h5py. Кроме того, данные в hdf позволяет сохранять библиотека для работы с табличными данными [pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_hdf.html)

HDF спроектирован для эффективного хранения многомерных массивов. 

Для примера работы получим три массива numpy и сохраним их в формат HDF.

In [5]:
from scipy.sparse import random
import numpy as np
from scipy.sparse.linalg import svds

# формируем массив случайных чисел
user_item_matrix = random(10000, 1000, density=0.01, format='coo', dtype=np.int32, random_state=42)
user_item_matrix = user_item_matrix.asfptype()
# разделяем массив на три части, каждую из которых будем хранить отдельно
U, s, V = svds(user_item_matrix, k=40)

print("Размерности матриц U={}, s={}, V={}".format(U.shape, s.shape, V.shape))
print("Типы матриц U={}, s={}, V={}".format(type(U), type(s), type(V)))

Размерности матриц U=(10000, 40), s=(40,), V=(40, 1000)
Типы матриц U=<class 'numpy.ndarray'>, s=<class 'numpy.ndarray'>, V=<class 'numpy.ndarray'>


Логической единицей хранения в hdf является dataset. Работа с датасетом не отличается от работы с текстовыми файлами, которые мы изучили ранее в курсе: мы создаём менеджер контекста **h5py.File** и загружаем датасет внутри контекста. За пределами менеджера контекста датасет доступен не будет

In [6]:
import h5py

with h5py.File('./data/s_matrix.hdf5', 'w') as f:
    h5py_dset = f.create_dataset("default", data=s)
    print('Создали датасет в менеджере контекста: {}'.format(h5py_dset))

print('Датасет вне менеджера контекста недоступен: {}'.format(h5py_dset))

Создали датасет в менеджере контекста: <HDF5 dataset "default": shape (40,), type "<f8">
Датасет вне менеджера контекста недоступен: <Closed HDF5 dataset>


Киллерфича hdf5 - сохранение датасет сложной структуры. Ближаший аналог - модуль pickle умеет сохранять только питоновские объекты, в то время как h5py способен хранить "разветвлённые" иерархические данные.

Сохраним три отдельных numpy.array в едином датасете:

In [7]:
with h5py.File('./data/complex_dataset.hdf5', 'w') as f:
    # создаём группу raw
    raw = f.create_group('source_data')
    raw.create_dataset('./data/complex_dataset.hdf5', data=np.random.random(1000))
    # создаём подгруппу processed
    processed = raw.create_group('model_data')
    # в подгруппе proceessed группы raw создаём датасет
    processed.create_dataset('user_factors', data=U, dtype=np.float32, compression="gzip")
    processed.create_dataset('eigen_values', data=s, dtype=np.float32, compression="gzip")
    processed.create_dataset('item_factors', data=V, dtype=np.float32, compression="gzip")
    
    print("Уровень группы:\t\t{}".format(raw.items()))
    print("Уровень подгруппы\t{}".format(processed.items()))
    # формируем список элементов, которые есть в подгруппе
    print("\nИмена элементов в подгруппе:\n")
    group_names = [i.name for i in f['source_data/model_data'].values()]
    for name in group_names:
        print(name)

Уровень группы:		ItemsViewHDF5(<HDF5 group "/source_data" (2 members)>)
Уровень подгруппы	ItemsViewHDF5(<HDF5 group "/source_data/model_data" (3 members)>)

Имена элементов в подгруппе:

/source_data/model_data/eigen_values
/source_data/model_data/item_factors
/source_data/model_data/user_factors


In [8]:
# для Windows
!where /R data /T *.hdf5

   1664366    9/7/2020    3:50:40 AM  D:\Љгабл\Skillbox\Data Science\18. —вҐ­ЁҐ Ё § ЇЁбм ¤ ­­ле з.2\data\complex_dataset.hdf5
      2368    9/7/2020    3:50:40 AM  D:\Љгабл\Skillbox\Data Science\18. —вҐ­ЁҐ Ё § ЇЁбм ¤ ­­ле з.2\data\s_matrix.hdf5


In [6]:
# для Linux
!ls -hla ./data | grep hdf5

-rw-r--r-- 1 adzhumurat adzhumurat 1,6M апр 14 19:20 complex_dataset.hdf5
-rw-r--r-- 1 adzhumurat adzhumurat 2,4K апр 14 19:20 s_matrix.hdf5


Ниже мы сохраням все результаты разложения в директорию `/data/`. Когда мы открываем датасет на чтение, он доступен для применения различных функций (вроде min, max), но при этом он теряет свойства numpy.ndarray. Чтобы перейти обратно к массиву можно сделать slice - мы итеративно пройдёмся по данным на диске и "соберём" их обратно в numpy.ndarray.

In [10]:
with h5py.File('./data/s_matrix.hdf5', 'r') as f:
    print("Доступные ключи %s\n" % list(f.keys()))
    data = f['default']
    print("min={}, \nmax={}, \nslice={}\n".format(min(data), max(data), data[10:15]))
    print("Попытка воспользоваться функциями numpy...")
    try:
        print(data.min())
    except AttributeError:
        print("Не вышло =(")
    data_copy = data[:]
print("Типизация датасетов: data={}, data_copy={}\n".format(data, type(data_copy)))

# Чтение иерархических датасетов
with h5py.File('./data/complex_dataset.hdf5', 'r') as f:
    U_hdf = f['source_data/model_data/user_factors'][:]
    s_hdf = f['source_data/model_data/eigen_values'][:]
    V_hdf = f['source_data/model_data/item_factors'][:]
    
print("Размерности матриц U={}, s={}, V={}".format(U_hdf.shape, s_hdf.shape, V_hdf.shape))
print("Типизация матриц U={}, s={}, V={}".format(type(U_hdf), type(s_hdf), type(V_hdf)))

Доступные ключи ['default']

min=15967970843.644163, 
max=16869307715.289244, 
slice=[1.61010982e+10 1.61161420e+10 1.61308403e+10 1.61512488e+10
 1.61722312e+10]

Попытка воспользоваться функциями numpy...
Не вышло =(
Типизация датасетов: data=<Closed HDF5 dataset>, data_copy=<class 'numpy.ndarray'>

Размерности матриц U=(10000, 40), s=(40,), V=(40, 1000)
Типизация матриц U=<class 'numpy.ndarray'>, s=<class 'numpy.ndarray'>, V=<class 'numpy.ndarray'>


## **18.4** *Работа с БД: SQLite3*

sqlite3 - либа для работы с реляционными СУБД. Реляционные СУБД представляют собой таблицы, которые соединяются друг с другом с помощью ключей. Ключ - это поле, значение которого в одной таблице совпадает со значением в другой таблице.

In [11]:
import sqlite3
import csv

conn = sqlite3.connect('./data/example.db') # создаём подключение к БД из дампа
c = conn.cursor() # специальный объект cursor, который служит для доступа к таблицам БД
print("Типизация connection {}, типизация cursor {}\n".format(type(conn), type(c)))

# Create table
c.execute('''DROP TABLE IF EXISTS jira_task''') # удаляем таблицу, если она уже есть в БД чтобы не дублировать данные
c.execute('''CREATE TABLE jira_task (code text, theme text, time_plan real, time_fact real)''') # создаём таблицу jira_task

with open('./data/task.csv', 'r', encoding='utf8') as fin: # т.н. менеджер контекста, аналогично уроку про чтение из файлов в python 
    # csv.DictReader использует первую строку текстового файла как заголовки столбцов по умолчанию
    dr = csv.reader(fin) # запятая - разделитель полей по умолчанию
    next(dr, None)  # пропускаем заголовок
    dataset = [(i[0], i[1], i[2], i[3]) for i in dr]

print("Выполняем INSERT в базу...")
c.executemany("INSERT INTO jira_task VALUES (?, ?, ? ,?);", dataset) # загрузка сформированного датасета в БД
print("Выполнили INSERT, закрываем соединение")
conn.commit() # функция commit сохраняет состояние БД
conn.close() # функция colse закрывает соединение

Типизация connection <class 'sqlite3.Connection'>, типизация cursor <class 'sqlite3.Cursor'>

Выполняем INSERT в базу...
Выполнили INSERT, закрываем соединение


После того, как таблицы созданы, можно читать из них данные

In [12]:
print("Открываем соединение с БД и читаем данные...\n")
conn = sqlite3.connect('./data/example.db')
c = conn.cursor()
some_row = None
for row in c.execute('SELECT * FROM jira_task LIMIT 10;'): # читаем первые 10 строк из БД
        print(row)
        some_row = row
conn.close()
print("\nТипизация строки %s" % type(row))

Открываем соединение с БД и читаем данные...

('HYDRA-535', 'Пробрасывать пользовательское распределение paid_types в ехидну', 'echidna', 1.0)
('HYDRA-534', 'Гибридный рекомендатель с multi-channel feedback', 'hydra', 3.0)
('HYDRA-532', 'Джоба в дженкинсе для расчёта динамики РВП', 'hydramatrices', 2.0)
('HYDRA-531', 'Интеграция Hydra с Gamora', 'hydramagrices', 4.0)
('HYDRA-530', 'Тестируем интеграцию с Jira', 'hydra', 2.0)
('HYDRA-527', 'Поправить функцию _get_ui_rec_matrix', 'hydra', 10.0)
('HYDRA-524', 'Оптимизировать матрицу ItemFactors', 'hydra', 2.0)
('HYDRA-523', 'Сортировка ЦПБ', 'hydra', 5.0)
('HYDRA-520', 'Закостылить параметр top', 'hydra', 2.0)
('HYDRA-519', "Сделать 'stable' конфигом по умолчанию в Гидре", 'hydra', 2.0)

Типизация строки <class 'tuple'>


## **18.5** *Работа с БД: PostgreSQL*

psycopg2 - библиотека для подключения к Postgres. Postgres - реляционная СУБД. Она содержит следующие сущности
* database - база данных
* table - таблица

Таблицы могут соединяться друг с другом с помощью операции **join** - то есть зависят друг от друга, поэтому такой тип хранилищ данных называется *реляционным*

Каждая таблица состоит из *кортежей* (или, по простому, строк). Данные из БД можно читать построчно.

Чтобы читать таблицу из БД, нужно авторизоваться с импользованием логина и пароля.

In [13]:
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

Выполним простейший запрос, который выведет все таблицы в базе *public*:

In [14]:
# запрос, который выведет все таблицы в базе public
sql_str = "SELECT table_name FROM information_schema.tables WHERE table_schema='public';"

cursor.execute(sql_str)
tables_data = [a for a in cursor.fetchall()] # метод fetchall возвращает все строки, которые удалось прочитать из запроса
conn.commit()

print("Какие таблицы содержатся в Postgres: %s" % tables_data)

Какие таблицы содержатся в Postgres: [('keywords',), ('links',), ('ratings',), ('exploratory',), ('course_purchases',), ('joi',)]


Postrgres - мощный аналитический инструмент, которорый позволяет перенести сложные расчёты из python на уровень ниже - в СУБД, которая хранит данные. СУБД представляет собой мощное вычислительное средство. Воспользуемся этим средством и сформируем аналитический запрос: посчитаем среднюю оценку по фильмам, у которых более 10 оценок и выведем top-5 таких фильмов с самой максимальной оценкой

In [15]:
import psycopg2

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)
cursor = conn.cursor()

# запрос
sql_str = """
    SELECT 
        movieId, AVG(rating) as avg_rating
    FROM public.ratings 
    GROUP BY movieId
    HAVING COUNT(RATING)>10
    ORDER BY avg_rating DESC
    LIMIT 10;
"""
cursor.execute(sql_str)


for row in cursor:
    print('movieId: %s %s avg_rating: %s' % (row[0], (7-len(str(row[0])))*' ', row[1]))
conn.close()
print("\nТипизация строки %s" % type(row))

movieId: 632      avg_rating: 4.63636363636364
movieId: 142115   avg_rating: 4.625
movieId: 99764    avg_rating: 4.58333333333333
movieId: 7096     avg_rating: 4.57142857142857
movieId: 4046     avg_rating: 4.55555555555556
movieId: 8724     avg_rating: 4.5
movieId: 111778   avg_rating: 4.5
movieId: 5169     avg_rating: 4.5
movieId: 4405     avg_rating: 4.5
movieId: 159817   avg_rating: 4.47826086956522

Типизация строки <class 'tuple'>


## **18.6** *Работа с БД из pandas*

Если в вашей БД хранится небольшое количество данных, которое помещается в оперативную память, то к базам можно подключатьться с помощью pandas. Для этих целей идеально подойдёт функция [pandas.read_sql_query](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql_query.html)

Прочитаем данные из sqlite

In [16]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("./data/example.db")
df = pd.read_sql_query("select * from jira_task limit 5;", conn)
df.head()

,code,theme,time_plan,time_fact
0,HYDRA-535,Пробрасывать пользовательское распределение pa...,echidna,1.0
1,HYDRA-534,Гибридный рекомендатель с multi-channel feedback,hydra,3.0
2,HYDRA-532,Джоба в дженкинсе для расчёта динамики РВП,hydramatrices,2.0
3,HYDRA-531,Интеграция Hydra с Gamora,hydramagrices,4.0
4,HYDRA-530,Тестируем интеграцию с Jira,hydra,2.0


Прочитаем данные из postgres

In [17]:
import psycopg2
import pandas as pd
import sqlite3

pg_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 5432,
    "dbname": "db_ds_students",
    "user": "readonly",
    "password": "6hajV34RTQfmxhS"
}
conn = psycopg2.connect(**pg_connection)

df = pd.read_sql_query("select * from public.ratings limit 5;", conn)
df.head()

,userid,movieid,rating,timestamp
0,4982,3793,5.0,995933596
1,4982,3826,3.0,995933278
2,4982,3863,4.0,995933094
3,4982,3949,5.0,995933663
4,4982,3977,3.0,995933094


## **18.7-18.8** *Работа с БД: MongoDB. Часть 1*

MongoDB - это т.н. **документоориентированная БД**, иногда их ещё называют бессхемными (schema-less).

В противоположность реляционным БД, где есть таблицы со строками, MongoDB оперирует другими понятиями:
* database - база данных, как в Postgres
* collection - коллекция, аналог таблицы
* document - документ, аналог строки в таблице

В отличие от реляционных БД, где все строки в таблице имеют одинаковые поля, каждый документ в коллекции MongoDB потенциально может иметь свой уникальный набор полей.
(Обычно так не делают. И большинство полей всё же пересекаются. Однако, в некоторых БД могут присутствовать поля, которых нет в других документах).

Подключёние происходит с помощью библиотеки **pymongo**, для подключения к БД нужно авторизоваться. Попробуем подключиться к БД и вытащить список доступных коллекций.

In [18]:
from pymongo import MongoClient

mongo_connection = {
    "host": "dsstudents.skillbox.ru",
    "port": 27017,
    "user": "students",
    "password": "X63673t47Gl03Sq",
    "authSource": "movies"
}

mongo = MongoClient('mongodb://%s:%s@%s:%s/?authSource=%s' % (
    mongo_connection['user'], mongo_connection['password'],
    mongo_connection['host'], mongo_connection['port'], mongo_connection['authSource'])
)
db = mongo["movies"]

print("Коллекции, доступные в MongoDB: %s" % db.list_collection_names())

Коллекции, доступные в MongoDB: ['tags', 'users']


MongroDB также является мощным вычислительным средством аналогично Postgre.

Можно посчитать самую простую статистику - количество документов в коллекции с помощью функции *.count()*

In [19]:
collection = db['tags']
print("Число документов в коллекции %s" % collection.estimated_document_count())

Число документов в коллекции 158680


In [20]:
mongo_cursor = collection.find().limit(5) # функция find - аналог оператора WHERE в SQL
print("Результат выборки: объект типа cursor %s\n" % mongo_cursor)

# пройдёмся по курсору и посмотрим, что внутри
cursor_items = [item for item in mongo_cursor]
print("Сожержимое курсора:\n%s\n" % cursor_items)
print("Поля элемента курсора %s" % list(cursor_items[0].keys()))

Результат выборки: объект типа cursor <pymongo.cursor.Cursor object at 0x0000029A89462C40>

Сожержимое курсора:
[{'_id': ObjectId('5c822402c0669da98bd5081e'), 'id': 931, 'name': 'jealousy'}, {'_id': ObjectId('5c822402c0669da98bd5081f'), 'id': 4290, 'name': 'toy'}, {'_id': ObjectId('5c822402c0669da98bd50820'), 'id': 5202, 'name': 'boy'}, {'_id': ObjectId('5c822402c0669da98bd50821'), 'id': 6054, 'name': 'friendship'}, {'_id': ObjectId('5c822402c0669da98bd50822'), 'id': 9713, 'name': 'friends'}]

Поля элемента курсора ['_id', 'id', 'name']


В метод *.find()* может принимать в том числе и аргументы. Например, т.н. *селектор*. *Селектор* - это словарь, который помогает оставить в выдачё только нужные поля. Селектор задаётся атрибутом *projection*

In [21]:
selector = {'name': True}
mongo_cursor = collection.find(projection=selector).limit(5)
cursor_items = [item for item in mongo_cursor]
print("Содержимое курсора (оставляем только поле 'name'):\n%s\n" % cursor_items)

# Нам мешается поле "_id" - можем выключить его
selector = {'_id': False}  
mongo_cursor = collection.find(projection=selector).limit(5)
cursor_items = [item for item in mongo_cursor]
print("Сожержимое курсора (выключаем _id):\n%s\n" % cursor_items)

#  можем выключить  поле "_id" и включить "name"
selector = {'_id': False, 'name': True}
mongo_cursor = collection.find(projection=selector).limit(5)
cursor_items = [item for item in mongo_cursor]
print("Сожержимое курсора без лишних полей:\n%s\n" % cursor_items)

Содержимое курсора (оставляем только поле 'name'):
[{'_id': ObjectId('5c822402c0669da98bd5081e'), 'name': 'jealousy'}, {'_id': ObjectId('5c822402c0669da98bd5081f'), 'name': 'toy'}, {'_id': ObjectId('5c822402c0669da98bd50820'), 'name': 'boy'}, {'_id': ObjectId('5c822402c0669da98bd50821'), 'name': 'friendship'}, {'_id': ObjectId('5c822402c0669da98bd50822'), 'name': 'friends'}]

Сожержимое курсора (выключаем _id):
[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}]

Сожержимое курсора без лишних полей:
[{'name': 'jealousy'}, {'name': 'toy'}, {'name': 'boy'}, {'name': 'friendship'}, {'name': 'friends'}]



### *Работа с БД: MongoDB. Часть 2*

Фильтровать можно с помощью параметра *.filter()*

In [22]:
selector = {'name': 'toy'}
exclude_id = {'_id': False}
mongo_cursor = collection.find(projection=exclude_id, filter={'name': 'toy'}).limit(5) # фильтр позволяет оставить только нужные нам тэги
cursor_items = [item for item in mongo_cursor]
print("Сожержимое курсора (оставляем только 'name'=='toy'):\n%s\n" % cursor_items)

Сожержимое курсора (оставляем только 'name'=='toy'):
[{'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}, {'id': 4290, 'name': 'toy'}]



MongoDB позволяет также выполнять сложные агрегирующие запросы средствами СУБД

In [23]:
pipline = [
    {"$group":            # этап группировки данных
        {"_id": "$name",  # гурппируем по полю name
         "tag_count":     # для каждого названия тэга
            {"$sum": 1}   # мы печатаем сумму вхождений этого тэга в коллекцию
         }
     },
    {"$sort":             # отсортируем все наши записи
        {"tag_count": -1} # по полю tag_count в обратном порядке (по убыванию)
     },
    {"$limit": 5}         # передаём модификатор limit (из выдачи оставим только 5 элементов)
]

print([i for i in collection.aggregate(pipline)])

[{'_id': 'woman director', 'tag_count': 3115}, {'_id': 'independent film', 'tag_count': 1930}, {'_id': 'murder', 'tag_count': 1308}, {'_id': 'based on novel', 'tag_count': 835}, {'_id': 'musical', 'tag_count': 734}]


Мы познакомились с нереляционным хранилищем данных *MongoDB*. *MongoDB* хранит в себе элементы, очень похожие на словари в Python. Поэтому формировать выдачу из словарей очень просто, и с помощью *MongoDB* можно строить сложные аналитические запросы, которые включают в себя аггрегацию данных и их сортировку.

## *Итоги*

В этом модуле мы познакомились с огромным количеством разных форматов данных:
- с очень распространённым текстовым форматом JSON
- с бинарными форматами данных, такими как pickle и более продвинутый формат для иерархического хранения данных - hdf5

Кроме того, мы узнали, что данные можно хранить не только на локальном компьютере в файлах или бинарных файлах, но и с помощью баз данных. В частности, мы познакомились с такими БД как:
- простая SQLite3
- более продвинутая PostreSQL
- нереляционная БД MongroDB